[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lawrennd/qig-code/blob/main/examples/generate-origin-paper-figures.ipynb)


# The Origin of the Inaccessible Game

## Neil D. Lawrence

## December 2025

This notebook provides comprehensive experiments that illustrate and validate.

1. LME state characterization (Lemma 3.1)
2. Qutrit optimality (Lemma 4.1)
3. Constraint linearization at origin (Lemma 3.2)
4. GENERIC-like decomposition structure
5. Entropy time reparametrization
6. Comparison of pair vs local operators
7. Global sufficiency of natural parameters
8. Trajectory dynamics from near-LME states

Each experiment produces figures and numerical results suitable for 
inclusion in the paper or supplementary materials.


In [ ]:
import sys
from pathlib import Path

# Add parent directory to path so we can import qig from examples/
_EXAMPLES_DIR = Path.cwd()
_PROJECT_ROOT = _EXAMPLES_DIR.parent
if str(_PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(_PROJECT_ROOT))


In [ ]:
# Auto-install TIG package if not available
import os

try:
    import qig
except ImportError:
    print("📦 Installing QIG package...")
    %pip install -q git+https://github.com/lawrennd/qig-code.git
    print("✓ QIG package installed!")

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
# Set plot style consistently
plt.style.use('seaborn-v0_8-whitegrid' if 'seaborn-v0_8-whitegrid' in plt.style.available else 'default')
big_wide_figsize = (10, 5)
big_figsize = (8, 8)
plt.rcParams.update({
    'font.size': 14,
    'font.family': 'serif',
    'axes.labelsize': 18,
    'xtick.labelsize': 16,
    'ytick.labelsize': 16,
    'legend.fontsize': 'medium',
})

# Create diagrams directory if it doesn't exist
os.makedirs('./diagrams', exist_ok=True)

print("✓ Plot configuration complete")
print("✓ Diagrams directory ready")

In [ ]:
from qig.exponential_family import QuantumExponentialFamily
from qig.dynamics import InaccessibleGameDynamics
from qig.core import von_neumann_entropy, marginal_entropies
from qig.pair_operators import bell_state_density_matrix, product_of_bell_states

# Experiment 1: LME State Characterization (Lemma 3.1)

Demonstrate that LME states achieve:
- Zero joint entropy H(ρ) = 0 (pure state)
- Maximal marginal entropies h_i = log(d_i)
- Maximum of Σh_i subject to purity

Figure shows: 
- Marginal entropies for LME vs separable states
- Multi-information I = Σh_i - H for various states



In [ ]:
import numpy as np


In [ ]:

results = {}

# Test for different dimensions
dimensions = [2, 3, 4, 5]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for idx, d in enumerate(dimensions):
    ax = axes.flatten()[idx]
    
    # Create LME state (Bell state for d-dimensional system)
    rho_lme = bell_state_density_matrix(d)
    
    # Compute entropies
    H_lme = von_neumann_entropy(rho_lme)
    h_lme = marginal_entropies(rho_lme, [d, d])
    C_lme = np.sum(h_lme)
    I_lme = C_lme - H_lme
    
    # Create separable pure state for comparison
    rho_sep = np.zeros((d**2, d**2), dtype=complex)
    rho_sep[0, 0] = 1.0  # |00⟩⟨00|
    H_sep = von_neumann_entropy(rho_sep)
    h_sep = marginal_entropies(rho_sep, [d, d])
    C_sep = np.sum(h_sep)
    I_sep = C_sep - H_sep
    
    # Create maximally mixed state
    rho_mixed = np.eye(d**2) / (d**2)
    H_mixed = von_neumann_entropy(rho_mixed)
    h_mixed = marginal_entropies(rho_mixed, [d, d])
    C_mixed = np.sum(h_mixed)
    I_mixed = C_mixed - H_mixed
    
    results[f'd={d}'] = {
        'LME': {'H': H_lme, 'h': h_lme, 'C': C_lme, 'I': I_lme},
        'Separable': {'H': H_sep, 'h': h_sep, 'C': C_sep, 'I': I_sep},
        'Mixed': {'H': H_mixed, 'h': h_mixed, 'C': C_mixed, 'I': I_mixed}
    }
    
    # Bar plot comparing marginal entropies
    x = np.arange(2)
    width = 0.25
    
    ax.bar(x - width, h_lme, width, label='LME', alpha=0.8, color='C0')
    ax.bar(x, h_sep, width, label='Separable |00⟩', alpha=0.8, color='C1')
    ax.bar(x + width, h_mixed, width, label='Maximally mixed', alpha=0.8, color='C2')
    
    ax.axhline(np.log(d), color='k', linestyle='--', linewidth=1, 
               alpha=0.5, label=f'log({d}) = {np.log(d):.3f}')
    
    ax.set_xlabel('Subsystem')
    ax.set_ylabel('Marginal entropy h_i')
    ax.set_title(f'd = {d} (qudits)')
    ax.set_xticks(x)
    ax.set_xticklabels(['Subsystem 1', 'Subsystem 2'])
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)
    
    print(f"d = {d}:")
    print(f"  LME state:      H = {H_lme:.6f}, Σh_i = {C_lme:.6f}, I = {I_lme:.6f}")
    print(f"  Separable |00⟩: H = {H_sep:.6f}, Σh_i = {C_sep:.6f}, I = {I_sep:.6f}")
    print(f"  Maximally mixed: H = {H_mixed:.6f}, Σh_i = {C_mixed:.6f}, I = {I_mixed:.6f}")
    print(f"  Max possible Σh_i: {2*np.log(d):.6f}")
    print()

plt.tight_layout()

plt.savefig('fig_exp1_lme_characterization.pdf', dpi=300, bbox_inches='tight')
print("✓ Saved: fig_exp1_lme_characterization.pdf")


# Experiment 2: Qutrit Optimality (Lemma 4.1)

Demonstrate that d=3 (qutrits) maximize the efficiency function
f(d) = (log d) / d under additive level-budget m = Σd_i.

Shows:
- Continuous optimization: d* = e ≈ 2.718
- Discrete optimization: d* = 3 (qutrits)
- Comparison of resource efficiency for d=2,3,4,5
    
    

In [ ]:
import numpy as np

In [ ]:
# Efficiency function f(d) = log(d)/d
def efficiency(d):
    return np.log(d) / d

# Continuous range
d_cont = np.linspace(1.5, 10, 1000)
f_cont = efficiency(d_cont)

# Discrete values
d_discrete = np.arange(2, 11)
f_discrete = efficiency(d_discrete)

# Optimal continuous value
d_optimal_cont = np.e
f_optimal_cont = efficiency(d_optimal_cont)

# Find optimal discrete value
d_optimal_discrete = d_discrete[np.argmax(f_discrete)]
f_optimal_discrete = efficiency(d_optimal_discrete)

print(f"Continuous optimization:")
print(f"  d* = e = {d_optimal_cont:.6f}")
print(f"  f(e) = {f_optimal_cont:.6f}")
print()
print(f"Discrete optimization (d ≥ 2):")
print(f"  d* = {d_optimal_discrete} (qutrits)")
print(f"  f(3) = {f_optimal_discrete:.6f}")
print()
print(f"Comparison:")
for d in [2, 3, 4, 5]:
    print(f"  d = {d}: f(d) = {efficiency(d):.6f}")
print()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Create figure
fig, ax = plt.subplots(figsize=big_wide_figsize)

# Panel 1: Efficiency function
ax.plot(d_cont, f_cont, 'b-', linewidth=2, label=r'$f(d) = \frac{\log d}{d}$')
ax.plot(d_discrete, f_discrete, 'ro', markersize=8, label='Integer d')
ax.axvline(d_optimal_cont, color='g', linestyle='--', linewidth=2, 
            alpha=0.7, label=f'd = e ≈ {d_optimal_cont:.3f} (continuous max)')
ax.axvline(d_optimal_discrete, color='r', linestyle='--', linewidth=2,
            alpha=0.7, label=f'd = {d_optimal_discrete} (discrete max)')

ax.set_xlabel(r'local dimension $d$')
ax.set_ylabel(r'efficiency $f(d) = \frac{\log d}{d}$')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim([1.5, 10])

plt.savefig('fig_exp2_resource-efficiency-vs-local-dimension.pdf', dpi=300, bbox_inches='tight')

In [ ]:
# Create figure
fig, ax = plt.subplots(figsize=big_wide_figsize)

# Panel 2: Entropy gradient factor C(d) = (m/d) * log(d) for fixed m
m_fixed = 12  # Example: 12 levels total

def entropy_potential(d, m):
    """Total entropy potential for n = m/d sites of dimension d."""
    if m % d != 0:
        return np.nan
    n = m // d
    return n * np.log(d)

# Find valid factorizations
valid_d = [d for d in range(2, m_fixed + 1) if m_fixed % d == 0]
C_values = [entropy_potential(d, m_fixed) for d in valid_d]

ax.bar(valid_d, C_values, color='steelblue', alpha=0.7, edgecolor='black')
ax.set_xlabel(r'local dimension $d$')
ax.set_ylabel(r'entropy potential $C(d) = \frac{m}{d} \log d$')
ax.set_xticks(valid_d)
ax.grid(True, alpha=0.3, axis='y')

# Highlight optimum
if 3 in valid_d:
    idx_3 = valid_d.index(3)
    ax.bar([3], [C_values[idx_3]], color='red', alpha=0.7, 
            edgecolor='black', label='d = 3 (optimal)')
    ax.legend()

print(f"For fixed budget m = {m_fixed}:")
for d, C in zip(valid_d, C_values):
    n = m_fixed // d
    print(f"  d = {d} (n = {n} sites): C = {C:.6f}")
print()

plt.tight_layout()

plt.savefig('fig_exp2_entropy-potential-for-fixed-budget.pdf', dpi=300, bbox_inches='tight')
print("✓ Saved: fig_exp2_qutrit_optimality.pdf")

results = {
    'd_optimal_continuous': d_optimal_cont,
    'd_optimal_discrete': d_optimal_discrete,
    'efficiency_values': {d: efficiency(d) for d in range(2, 11)}
}

# Experiment 3: Constraint Gradient at Origin (Lemma 3.2)

Demonstrate that the constraint gradient ∇(Σh_i) vanishes at LME states.

Shows:

- Gradient norm ||∇C|| vs distance from LME state
- Constraint linearization quality near origin
- Comparison for different dimensions d
 

In [ ]:
   
results = {}

# Test for d=2,3

for idx, d in enumerate([2, 3]):
    fig, ax = plt.subplots(figsize=big_figsize)
    
    # Create exponential family with pair operators
    print(f"Testing d = {d}...")
    exp_fam = QuantumExponentialFamily(n_pairs=1, d=d, pair_basis=True)
    
    # LME state corresponds to theta near zero in pair basis
    # (Since Bell state has maximum marginal entropies)
    theta_lme = np.zeros(exp_fam.n_params)
    
    # Compute constraint gradient at LME
    C_lme, grad_lme = exp_fam.marginal_entropy_constraint(theta_lme)
    grad_norm_lme = np.linalg.norm(grad_lme)
    
    print(f"  At LME (θ=0): ||∇C|| = {grad_norm_lme:.10f}")
    print(f"  Constraint value C = {C_lme:.6f}")
    print(f"  Maximum possible: {2*np.log(d):.6f}")
    print()
    
    # Compute gradient along radial perturbations
    epsilon_values = np.logspace(-3, 0, 20)  # Perturbation magnitudes
    grad_norms = []
    constraint_values = []
    
    # Random direction in parameter space
    np.random.seed(42)
    direction = np.random.randn(exp_fam.n_params)
    direction = direction / np.linalg.norm(direction)
    
    for eps in epsilon_values:
        theta_pert = theta_lme + eps * direction
        try:
            C_pert, grad_pert = exp_fam.marginal_entropy_constraint(theta_pert)
            grad_norms.append(np.linalg.norm(grad_pert))
            constraint_values.append(C_pert)
        except:
            grad_norms.append(np.nan)
            constraint_values.append(np.nan)
    
    # Plot gradient norm vs perturbation
    ax.loglog(epsilon_values, grad_norms, 'bo-', linewidth=2, 
              markersize=6, label='||∇C(θ)||')
    ax.axhline(grad_norm_lme, color='r', linestyle='--', linewidth=2,
               alpha=0.7, label=f'||∇C(0)|| = {grad_norm_lme:.2e}')
    
    # Expected linear scaling near origin
    ax.loglog(epsilon_values, epsilon_values * grad_norms[10], 'g--', 
              linewidth=1.5, alpha=0.7, label='O(ε) scaling')
    
    ax.set_xlabel(r'perturbation magnitude $\epsilon$')
    ax.set_ylabel(r'constraint gradient norm $\|\nabla C\|$')
    ax.legend()
    ax.grid(True, alpha=0.3, which='both')
    
    results[f'd={d}'] = {
        'grad_norm_at_lme': grad_norm_lme,
        'C_lme': C_lme,
        'grad_norms_perturbed': grad_norms,
        'epsilon_values': epsilon_values
    }

    plt.tight_layout()

    plt.savefig(f'fig_exp3_constraint_entangled-pair-with-d-{d}.pdf', dpi=300, bbox_inches='tight')


# Experiment 4: GENERIC Decomposition

Demonstrate GENERIC-like decomposition of the Jacobian M = S + A
where S is symmetric (dissipative) and A is antisymmetric (conservative).

Shows:
- Decomposition quality along trajectory
- Relative magnitudes of ||S|| and ||A||
- Degeneracy conditions: S·∇H = flow, A·∇H = 0
    
    

In [ ]:
# Create qutrit pair system
d = 3
exp_fam = QuantumExponentialFamily(n_pairs=1, d=d, pair_basis=True)

# Initialize near (but not at) LME
np.random.seed(42)
theta_init = 0.1 * np.random.randn(exp_fam.n_params)

print(f"Running dynamics for {d}×{d} system...")
print(f"Initial parameters: ||θ|| = {np.linalg.norm(theta_init):.4f}")

# Run short trajectory
dynamics = InaccessibleGameDynamics(exp_fam, method='sld')
result = dynamics.solve_constrained_maxent(
    theta_init, 
    n_steps=500,
    dt=0.005,
    convergence_tol=1e-5,
    project=True,
    project_every=10
)

print(f"✓ Converged: {result['converged']}")
print(f"✓ Steps: {result['n_steps']}")
print()

# Sample points along trajectory
trajectory = result['trajectory']
sample_indices = np.linspace(0, len(trajectory)-1, 20, dtype=int)

# Compute GENERIC decomposition at each point
print("Computing GENERIC decomposition at sampled points...")
norms_M = []
norms_S = []
norms_A = []
ratios = []
symmetry_errors = []
antisymmetry_errors = []

for i in sample_indices:
    theta = trajectory[i]
    
    # Compute Jacobian M
    M = exp_fam.jacobian(theta, method='sld')
    
    # Decompose into symmetric and antisymmetric parts
    S = 0.5 * (M + M.T)
    A = 0.5 * (M - M.T)
    
    # Compute norms
    norms_M.append(np.linalg.norm(M, 'fro'))
    norms_S.append(np.linalg.norm(S, 'fro'))
    norms_A.append(np.linalg.norm(A, 'fro'))
    
    # Ratio of antisymmetric to symmetric
    if norms_S[-1] > 1e-10:
        ratios.append(norms_A[-1] / norms_S[-1])
    else:
        ratios.append(np.nan)
    
    # Check decomposition quality
    symmetry_errors.append(np.linalg.norm(S - S.T, 'fro'))
    antisymmetry_errors.append(np.linalg.norm(A + A.T, 'fro'))

print(f"Mean ||S||: {np.nanmean(norms_S):.6f}")
print(f"Mean ||A||: {np.nanmean(norms_A):.6f}")
print(f"Mean ||A||/||S||: {np.nanmean(ratios):.6f}")
print()

results = {
    'trajectory': trajectory,
    'sample_indices': sample_indices,
    'norms_S': norms_S,
    'norms_A': norms_A,
    'ratios': ratios
}

In [ ]:
norms_S

In [ ]:
# Create figure
fig, ax = plt.subplots(figsize=big_figsize)

# Panel 1: Component norms
ax.plot(sample_indices, norms_S, 'r-o', linewidth=4, markersize=6,
        label=r'$\|S\|$ (symmetric/dissipative)')
ax.plot(sample_indices, norms_A, 'b-s', linewidth=4, markersize=6,
        label=r'$\|A\|$ (antisymmetric/conservative)')
ax.plot(sample_indices, norms_M, 'k--', linewidth=3, alpha=0.5,
        label=r'$\|M\|$ (total Jacobian)')
ax.set_xlabel('trajectory step')
ax.set_ylabel('frobenius norm')
ax.legend()
ax.grid(True, alpha=0.3)

plt.savefig('fig_exp4_generic-component-maginitude.pdf', dpi=300, bbox_inches='tight')


In [ ]:
fig, ax = plt.subplots(figsize=big_figsize)

ax.plot(sample_indices, ratios, 'g-o', linewidth=2, markersize=6)
ax.set_xlabel('trajectory step')
ax.set_ylabel(r'ratio $\|A\|/\|S\|$')
ax.grid(True, alpha=0.3)
ax.axhline(1.0, color='k', linestyle='--', linewidth=1, alpha=0.5,
           label='Equal magnitudes')
ax.legend()

plt.savefig('fig_exp4_conservative-vs-dissipative-balance.pdf', dpi=300, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=big_figsize)

ax.semilogy(sample_indices, symmetry_errors, 'r-o', linewidth=2, markersize=6,
            label=r'$\|S - S^T\|$ (should be 0)')
ax.semilogy(sample_indices, antisymmetry_errors, 'b-s', linewidth=2, markersize=6,
            label=r'$\|A + A^T\|$ (should be 0)')
ax.set_xlabel('trajectory step')
ax.set_ylabel('Frobenius norm (log scale)')
ax.legend()
ax.grid(True, alpha=0.3, which='both')

plt.savefig('fig_exp4_decomposition-quality.pdf', dpi=300, bbox_inches='tight')

In [ ]:
# Panel 4: Flow norms
fig, ax = plt.subplots(figsize=big_figsize)

ax.semilogy(result['flow_norms'], 'purple', linewidth=4)
ax.set_xlabel('trajectory step')
ax.set_ylabel(r'$\|F(\theta)\|$ (log scale)')
ax.grid(True, alpha=0.3, which='both')
ax.axhline(1e-5, color='r', linestyle='--', linewidth=2,
           alpha=0.5, label='Convergence threshold')
ax.legend()

plt.tight_layout()

plt.savefig('fig_exp4_flow-magnitude-convergence.pdf', dpi=300, bbox_inches='tight')



# Experiment 5: Entropy Time Reparametrization

Demonstrate entropy time reparametrization that removes singularity.

Shows:

- Entropy production dH/dτ in affine time (diverges at origin)
- Entropy production dH/dt in entropy time (constant = 1)
- Time dilation factor dτ/dt along trajectory

In [ ]:
# Create system
d = 3
exp_fam = QuantumExponentialFamily(n_pairs=1, d=d, pair_basis=True)

# Initialize at relatively mixed state
np.random.seed(42)
theta_init = 0.3 * np.random.randn(exp_fam.n_params)

print(f"Running dynamics for {d}×{d} system...")

# Run trajectory
dynamics = InaccessibleGameDynamics(exp_fam, method='sld')
result = dynamics.solve_constrained_maxent(
    theta_init,
    n_steps=1000,
    dt=0.005,
    convergence_tol=1e-5,
    project=True,
    project_every=10
)

print(f"✓ Converged: {result['converged']}")
print()

# Compute entropy along trajectory
trajectory = result['trajectory']
H_traj = []
theta_norm_traj = []

print("Computing entropy production rates...")
for theta in trajectory:
    H_traj.append(exp_fam.von_neumann_entropy(theta))
    theta_norm_traj.append(np.linalg.norm(theta))

H_traj = np.array(H_traj)
theta_norm_traj = np.array(theta_norm_traj)

# Compute entropy production in affine time dH/dτ
dt_affine = 0.005
dH_dtau = np.gradient(H_traj) / dt_affine

# Compute entropy time: t = ∫ (dH/dτ) dτ
entropy_time = np.cumsum(dH_dtau) * dt_affine

# Time dilation factor dτ/dt = 1/(dH/dτ)
time_dilation = np.where(dH_dtau > 1e-10, 1.0 / dH_dtau, np.nan)

results = {
    'H_trajectory': H_traj,
    'entropy_production_affine': dH_dtau,
    'entropy_time': entropy_time,
    'time_dilation': time_dilation,
    'theta_norm': theta_norm_traj
}
print(f"Initial entropy: H(0) = {H_traj[0]:.6f}")
print(f"Final entropy: H(end) = {H_traj[-1]:.6f}")
print(f"Total entropy increase: ΔH = {H_traj[-1] - H_traj[0]:.6f}")
print(f"Mean dH/dτ: {np.nanmean(dH_dtau):.6f}")
print()

In [ ]:
fig, ax = plt.subplots(figsize=big_figsize)

# Panel 1: Entropy vs affine time
ax.plot(H_traj, 'b-', linewidth=2)
ax.set_xlabel(r'affine time step $\tau$')
ax.set_ylabel(r'joint entropy $H(\theta(\tau))$')
ax.grid(True, alpha=0.3)
plt.savefig('fig_exp5_entropy-evolution-affine-time.pdf', dpi=300, bbox_inches='tight')


In [ ]:
# Panel 2: Entropy production rate in affine time
fig, ax = plt.subplots(figsize=big_figsize)

ax.plot(dH_dtau, 'r-', linewidth=2)
ax.set_xlabel(r'Affine time step $\tau$')
ax.set_ylabel(r'$\frac{\text{d}H}{\text{d}\tau}$')
ax.grid(True, alpha=0.3)
ax.axhline(0, color='k', linestyle='-', linewidth=0.5)
plt.savefig('fig_exp5_entropy-production-rate-affine-time.pdf', dpi=300, bbox_inches='tight')


In [ ]:
# Panel 3: Entropy vs entropy time (should be linear with slope 1)
fig, ax = plt.subplots(figsize=big_figsize)

# Only plot where entropy time is well-defined
valid_mask = ~np.isnan(entropy_time) & (entropy_time > 0)
ax.plot(entropy_time[valid_mask], H_traj[valid_mask], 'g-', linewidth=2,
        label=r'$H(t)$')
# Reference line with slope 1
t_ref = np.linspace(entropy_time[valid_mask][0], 
                    entropy_time[valid_mask][-1], 100)
H_ref = H_traj[valid_mask][0] + (t_ref - entropy_time[valid_mask][0])
ax.plot(t_ref, H_ref, 'k--', linewidth=1.5, alpha=0.5,
        label='slope = 1 (ideal)')
ax.set_xlabel(r'entropy time $t$')
ax.set_ylabel(r'joint entropy $H(\theta(t))$')
ax.legend()
ax.grid(True, alpha=0.3)
plt.savefig('fig_exp5_entropy-evolution-entropy-time.pdf', dpi=300, bbox_inches='tight')


In [ ]:
# Panel 4: Time dilation factor
fig, ax = plt.subplots(figsize=big_figsize)

ax.semilogy(theta_norm_traj, time_dilation, 'v-', 
            linewidth=2, markersize=4, alpha=0.7)
ax.set_xlabel(r'parameter norm $\|\theta\|$')
ax.set_ylabel(r'time dilation $\frac{d\tau}{dt} = \frac{1}{dH/d\tau}$')
ax.set_title('Time Dilation Factor')
ax.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.savefig('fig_exp5_parameter-norm-time-dilation.pdf', dpi=300, bbox_inches='tight')


# Experiment 6: Pair vs Local Operators

Compare pair operators (can create entanglement) vs local operators
(can only create separable states).

Shows:

- Mutual information I = C - H for both cases
- Constraint manifold structure differences
- Why pair operators are necessary for quantum game
    
    

In [ ]:
d = 3

# Case 1: Pair operators (can create entanglement)
print("Creating system with PAIR operators (entangling)...")
exp_fam_pair = QuantumExponentialFamily(n_pairs=1, d=d, pair_basis=True)

# Start at LME (maximum entanglement)
theta_lme = np.zeros(exp_fam_pair.n_params)
rho_lme = exp_fam_pair.rho_from_theta(theta_lme)

H_lme = exp_fam_pair.von_neumann_entropy(theta_lme)
I_lme = exp_fam_pair.mutual_information(theta_lme)

print(f"  LME state (pair basis):")
print(f"    H = {H_lme:.6f}, I = {I_lme:.6f}")
print()

# Case 2: Local operators (cannot create entanglement)
print("Creating system with LOCAL operators (non-entangling)...")
exp_fam_local = QuantumExponentialFamily(n_sites=2, d=d, pair_basis=False)

# Try various parameter settings with local operators
print("  Testing various local parameter settings...")
I_values_local = []
H_values_local = []

np.random.seed(42)
for trial in range(20):
    theta_local = np.random.randn(exp_fam_local.n_params) * (0.1 + 0.4 * trial/20)
    rho_local = exp_fam_local.rho_from_theta(theta_local)
    
    # Compute entropies
    H_local = von_neumann_entropy(rho_local)
    h_local = marginal_entropies(rho_local, exp_fam_local.dims)
    C_local = np.sum(h_local)
    I_local = C_local - H_local
    
    I_values_local.append(I_local)
    H_values_local.append(H_local)

max_I_local = np.max(I_values_local)

results = {
    'pair_operators': {
        'I_lme': I_lme,
        'H_lme': H_lme,
        'can_entangle': True
    },
    'local_operators': {
        'I_max': max_I_local,
        'H_values': H_values_local,
        'I_values': I_values_local,
        'can_entangle': False
    }
}
print(f"    Maximum I with local operators: {max_I_local:.10f}")
print(f"    (Should be ≈ 0 due to separability)")
print()



In [ ]:
# Panel 1: Mutual information comparison
# Create comparison figure
fig, ax = plt.subplots(figsize=big_figsize)

# Bar chart
categories = ['Pair operators\n(at LME)', 'Local operators\n(maximum found)']
I_comparison = [I_lme, max_I_local]
colors = ['steelblue', 'coral']

ax.bar(categories, I_comparison, color=colors, alpha=0.7, edgecolor='black',
       linewidth=2)
ax.set_ylabel(r'Mutual information $I = C - H$')
ax.set_title('Entanglement Capability Comparison')
ax.grid(True, alpha=0.3, axis='y')

# Add numerical labels
for i, (cat, val) in enumerate(zip(categories, I_comparison)):
    ax.annotate(f'{val:.4f}', xy=(i, val), xytext=(0, 5), 
                textcoords='offset points', ha='center', fontsize=12, fontweight='bold')


In [ ]:
# Panel 2: Scatter plot of achievable (H, I) points
fig, ax = plt.subplots(figsize=big_figsize)

# Local operators: scatter of (H, I) points
ax.scatter(H_values_local, I_values_local, c='coral', s=100, alpha=0.6,
           edgecolors='darkred', linewidth=1.5, label='Local operators')

# Pair operators: LME point
ax.scatter([H_lme], [I_lme], c='steelblue', s=300, alpha=0.8,
           edgecolors='darkblue', linewidth=2, marker='*',
           label='Pair operators (LME)', zorder=10)

ax.set_xlabel(r'joint entropy $H$')
ax.set_ylabel(r'mutual information $I$')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Add constraint line I + H = C
C_max = 2 * np.log(d)
H_line = np.linspace(0, C_max, 100)
I_line = C_max - H_line
ax.plot(H_line, I_line, 'k--', linewidth=1.5, alpha=0.5,
        label=f'I + H = {C_max:.3f} (max constraint)')
ax.legend(fontsize=9)

plt.tight_layout()

plt.savefig('fig_exp6_achievable-states-in-H-I-space.pdf', dpi=300, bbox_inches='tight')



# Experiment 7: Global Sufficiency (Theorem 3.4)

Demonstrate that the operator basis chosen at origin remains sufficient
throughout dynamics (no reparametrisation needed).

Shows:

- Tangent space coverage along trajectory
- Metric conditioning throughout manifold
- No coordinate singularities away from boundary
    


In [ ]:
# Create system
d = 3
exp_fam = QuantumExponentialFamily(n_pairs=1, d=d, pair_basis=True)

# Run trajectory from random start
np.random.seed(24)
theta_init = exp_fam.get_bell_state_parameters(epsilon=1e-6)


print("Running dynamics...")
dynamics = InaccessibleGameDynamics(exp_fam)
result = dynamics.solve_constrained_maxent(
    theta_init,
    n_steps=800,
    dt=0.005,
    convergence_tol=1e-5,
    project=True,
    project_every=10
)


# Sample trajectory
trajectory = result['trajectory']
sample_indices = np.linspace(0, len(trajectory)-1, 30, dtype=int)

# Compute Fisher metric properties at each point
print("Computing metric properties along trajectory...")
eigenvalue_ratios = []  # Condition number = λ_max / λ_min
det_G = []  # Determinant of Fisher metric
H_values = []

for i in sample_indices:
    theta = trajectory[i]
    
    # Compute Fisher information
    G = exp_fam.fisher_information(theta)
    
    # Eigenvalues
    eigvals = np.linalg.eigvalsh(G)
    eigvals = eigvals[eigvals > 1e-12]  # Remove numerical zeros
    
    if len(eigvals) > 0:
        cond_number = eigvals[-1] / eigvals[0]
        eigenvalue_ratios.append(cond_number)
        det_G.append(np.prod(eigvals))
    else:
        eigenvalue_ratios.append(np.nan)
        det_G.append(np.nan)
    
    # Entropy
    H_values.append(exp_fam.von_neumann_entropy(theta))

eigenvalue_ratios = np.array(eigenvalue_ratios)
det_G = np.array(det_G)
H_values = np.array(H_values)

results = {
    'condition_numbers': eigenvalue_ratios,
    'determinants': det_G,
    'entropy_values': H_values,
    'trajectory_indices': sample_indices
}
print(f"Metric condition numbers:")
print(f"  Mean: {np.nanmean(eigenvalue_ratios):.3e}")
print(f"  Std: {np.nanstd(eigenvalue_ratios):.3e}")
print(f"  Min: {np.nanmin(eigenvalue_ratios):.3e}")
print(f"  Max: {np.nanmax(eigenvalue_ratios):.3e}")
print()
print("Interpretation: Condition number O(10^1 - 10^3) indicates well-conditioned")
print("metric throughout trajectory. No reparametrisation needed!")
print()

In [ ]:
# Create figure
fig, ax = plt.subplots(figsize=big_figsize)

# Panel 1: Condition number vs trajectory step
ax.semilogy(sample_indices, eigenvalue_ratios, 'bo-', linewidth=4, 
            markersize=6)
ax.set_xlabel('trajectory step')
ax.set_ylabel(r'Condition number $\lambda_{\max}/\lambda_{\min}$')
ax.set_title('Metric Conditioning (Lower = Better)')
ax.grid(True, alpha=0.3, which='both')
ax.axhline(1e6, color='r', linestyle='--', linewidth=3, alpha=0.7,
           label='Poor conditioning threshold')
ax.legend()

plt.tight_layout()
plt.savefig('fig_exp7_metric-conditioning.pdf', dpi=300, bbox_inches='tight')


In [ ]:
# Panel 2: Condition number vs entropy
fig, ax = plt.subplots(figsize=big_figsize)

ax.semilogy(H_values, eigenvalue_ratios, 'go-', linewidth=4, markersize=6)
ax.set_xlabel(r'joint entropy $H$')
ax.set_ylabel(r'condition number $\lambda_{\max}/\lambda_{\min}$')
ax.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.savefig('fig_exp7_metric-conditioning-vs-entropy.pdf', dpi=300, bbox_inches='tight')


In [ ]:
# Panel 3: Determinant of metric
fig, ax = plt.subplots(figsize=big_figsize)

ax.semilogy(sample_indices, det_G, 'ro-', linewidth=4, markersize=6)
ax.set_xlabel('trajectory step')
ax.set_ylabel(r'$\det(G)$')
ax.grid(True, alpha=0.3, which='both')
plt.tight_layout()
plt.savefig('fig_exp7_metric-volume-element.pdf', dpi=300, bbox_inches='tight')


In [ ]:
# Panel 4: Entropy trajectory
fig, ax = plt.subplots(figsize=big_figsize)

ax.plot(sample_indices, H_values, 'v-', linewidth=4)
ax.set_xlabel('trajectory step')
ax.set_ylabel('joint entropy $H$')
ax.grid(True, alpha=0.3)
ax.axhline(0, color='b', linestyle='--', linewidth=1, alpha=0.5,
           label=r'$H = 0$ (pure state)')
ax.axhline(2*np.log(d), color='r', linestyle='--', linewidth=2, alpha=0.5,
           label=fr'$H = {2*np.log(d):.3f}$ (maximally mixed)')
ax.legend()

plt.tight_layout()
plt.savefig('fig_exp7_entropy-evolution.pdf', dpi=300, bbox_inches='tight')



# Experiment 8: Dimension Comparison

Compare dynamics for different local dimensions d=2,3,4 to illustrate
why qutrits (d=3) are optimal.

Shows:

- Initial gradient factors for each dimension
- Convergence rates
- Entropy production profiles

    

In [ ]:
dimensions = [2, 3, 4]
results = {}

for d in dimensions:
    print(f"Running dynamics for d = {d}...")
    
    # Create system
    exp_fam = QuantumExponentialFamily(n_pairs=1, d=d, pair_basis=True)
    
    # Initialize near LME
    np.random.seed(42)
    theta_init = np.random.randn(exp_fam.n_params)
    
    # Compute initial entropy gradient factor
    G_init = exp_fam.fisher_information(theta_init)
    _, a_init = exp_fam.marginal_entropy_constraint(theta_init)
    
    # Projection matrix
    a_norm_sq = np.dot(a_init, a_init)
    if a_norm_sq > 1e-12:
        Pi = np.eye(len(theta_init)) - np.outer(a_init, a_init) / a_norm_sq
    else:
        Pi = np.eye(len(theta_init))
    
    # Entropy production rate (affine time)
    entropy_grad_factor = theta_init @ G_init @ Pi @ G_init @ theta_init
    
    print(f"  Initial entropy gradient factor: {entropy_grad_factor:.6e}")
    
    # Run dynamics
    dynamics = InaccessibleGameDynamics(exp_fam)
    result = dynamics.solve_constrained_maxent(
        theta_init,
        n_steps=600,
        dt=0.005,
        convergence_tol=1e-5,
        project=True,
        project_every=10
    )
    
    print(f"  Converged: {result['converged']}, Steps: {result['n_steps']}")
    print()
    
    # Compute entropy trajectory
    H_traj = []
    for theta in result['trajectory']:
        H_traj.append(exp_fam.von_neumann_entropy(theta))
    H_traj = np.array(H_traj)
    
    results[f'd={d}'] = {
        'entropy_gradient_factor': entropy_grad_factor,
        'trajectory': result['trajectory'],
        'H_trajectory': H_traj,
        'flow_norms': result['flow_norms'],
        'converged': result['converged']
    }

print("\nSummary of entropy gradient factors:")
for d in dimensions:
    factor = results[f'd={d}']['entropy_gradient_factor']
    print(f"  d = {d}: {factor:.6e}")

In [ ]:
fig, ax = plt.subplots(figsize=big_figsize)

# Plot entropy trajectories
for d in dimensions:
    ax.plot(results[f'd={d}']['H_trajectory'], linewidth=4, label=f'd = {d}')
    
# Finalize Panel 1    
ax.set_xlabel('trajectory step')
ax.set_ylabel(r'joint entropy $H$')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('fig_exp8_entropy-evolution-for-different-dimensions.pdf', dpi=300, bbox_inches='tight')

    

In [ ]:
fig, ax = plt.subplots(figsize=big_figsize)

# Plot flow norms
for d in dimensions:
    ax.semilogy(results[f'd={d}']['flow_norms'], linewidth=4, label=f'd = {d}')
# Finalize Panel 2
ax.set_xlabel('Trajectory step')
ax.set_ylabel(r'$\|F(\theta)\|$ (log scale)')
ax.legend()
ax.grid(True, alpha=0.3, which='both')
ax.axhline(1e-5, color='k', linestyle='--', linewidth=2, alpha=0.5,
           label='Threshold')

plt.tight_layout()
plt.savefig('fig_exp8_convergence-rate-comparison.pdf', dpi=300, bbox_inches='tight')


In [ ]:
fig, ax = plt.subplots(figsize=big_figsize)

# Plot initial entropy production factor
for d in dimensions:
    ax.bar(d, results[f'd={d}']['entropy_gradient_factor'], width=0.6, alpha=0.7)

# Finalize Panel 3
ax.set_xlabel('Local dimension d')
ax.set_ylabel(r'Initial $\theta^\top G \Pi G \theta$')
ax.set_xticks(dimensions)
ax.grid(True, alpha=0.3, axis='y')

# Add optimal marker
factors = [results[f'd={d}']['entropy_gradient_factor'] for d in dimensions]
optimal_idx = np.argmax(factors)
ax.bar([dimensions[optimal_idx]], [factors[optimal_idx]], 
       width=0.6, alpha=0.7, color='red', label='maximum')
ax.legend()

plt.tight_layout()
plt.savefig('fig_exp8_entropy-gradient-factor-at-start.pdf', dpi=300, bbox_inches='tight')


In [ ]:
fig, ax = plt.subplots(figsize=big_figsize)

# Plot parameter norm evolution
for d in dimensions:
    theta_norms = [np.linalg.norm(theta) for theta in results[f'd={d}']['trajectory']]
    ax.plot(theta_norms, linewidth=4, label=f'd = {d}')

# Finalize Panel 4
ax.set_xlabel('Trajectory step')
ax.set_ylabel(r'$\|\theta\|$')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('fig_exp8_parameter-norm-evolution.pdf', dpi=300, bbox_inches='tight')
